In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

os.chdir('C:/Users/kmes8/Desktop/大數據與商業分析/91APP_DataSet_2022')

## 依照參數回傳每人的資料 
輸入格式(資料名(combined), 年, 季(1 -> 1~3月 ...))

In [2]:
def get_season(combined, y, s):
    # if s == 4:
    #     lower = datetime.datetime(y,s*3-2,1,0,0)
    #     upper = datetime.datetime(y+1,1,1,0,0)
    # elif s < 4 and s > 0:
    #     lower = datetime.datetime(y,s*3-2,1,0,0)
    #     upper = datetime.datetime(y,s*3+1,1,0,0)
    # else:
    #     print('s must be 1, 2, 3 or 4')
    #     return 0
    if s == 4:
        lower = str(y)+'-10-01'
        upper = str(y+1)+'-01-01'
    elif s == 3:
        lower = str(y)+'-07-01'
        upper = str(y)+'-10-01'
    elif s == 1:
        lower = str(y)+'-01-01'
        upper = str(y)+'-04-01'
    elif s == 2:
        lower = str(y)+'-04-01'
        upper = str(y)+'-07-01'
    else:
        print('s must be 1, 2, 3 or 4')
        return 0
    print(lower, upper)
    
    ids = combined['UnifiedUserId'].unique()
    combined = combined[(combined['OrderDateTime'] < upper) & (combined['OrderDateTime'] > lower)]
  
    lst = []
    for i in ids:
        ele = {'UnifiedUserId': i,
               'MemberCardLevel': 0,
               'Gender': 'Ungiven',
               'Age': 0,
               'order': 0, #本季主單數
               'orderSlave': 0, #本季子單數
               'low': 0, #本季低價位商品購買數
               'normal': 0, #本季中價位商品購買數
               'high': 0, #本季高價位商品購買數
               'sales': 0,# 本季總消費
               'SalePageId':""
                }
        lst.append(ele)
        
    re = pd.DataFrame(lst).set_index('UnifiedUserId').copy()
    
    for index, r in combined.iterrows():
        idx = r['UnifiedUserId']
        re.loc.__setitem__((idx, ('Gender')), r['Gender'])
        re.loc.__setitem__((idx, ('MemberCardLevel')), r['MemberCardLevel'])
        re.loc.__setitem__((idx, ('Age')), int((datetime.datetime.today() - datetime.datetime.strptime(r['Birthday'], '%Y-%m-%d')).days/365))
        re.loc.__setitem__((idx, ('orderSlave')), r['TsCount'] + re.loc[idx]['orderSlave'])
        re.loc.__setitem__((idx, ('order')), 1 + re.loc[idx]['order'])
        re.loc.__setitem__((idx, ('sales')), r['TotalSalesAmount'] + re.loc[idx]['sales'])
        re.loc.__setitem__((idx, ('high')), r['high'] + re.loc[idx]['high'])
        re.loc.__setitem__((idx, ('normal')), r['normal'] + re.loc[idx]['normal'])
        re.loc.__setitem__((idx, ('low')), r['low'] + re.loc[idx]['low'])
        re.loc.__setitem__((idx, ('SalePageId')), r['SalePageId'] + "/" + re.loc[idx]['SalePageId'])
        
        

    re['avgSales'] = re['sales']/re['order']
                
                
    return re

## 讀取資料

In [3]:
member = pd.read_csv('91APP_MemberData.csv')
order = pd.read_csv('91APP_OrderData.csv')
product = pd.read_excel("user_tag.xlsx").dropna()

C:\Users\kmes8\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
member = member[['UnifiedUserId', 'Gender', 'Birthday', 'MemberCardLevel']]
member.head(3)

,UnifiedUserId,Gender,Birthday,MemberCardLevel
0,wDnLYnQDE5Nt/TBttarrEw==,Female,1987-11-12,10
1,PfTmZ2HBNiyYJrv0kLZpnw==,Female,1990-09-02,10
2,MajPkebcNSUBGwNtJOjVMA==,Female,1985-09-08,10


In [6]:
order = order[['TradesGroupCode', 'OrderDateTime', 'TsCount', 'Qty', 'TotalSalesAmount']]
order.head(3)

,TradesGroupCode,OrderDateTime,TsCount,Qty,TotalSalesAmount
0,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,1,-1,-100.0
1,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,2,2,1284.0
2,VTI/4oAuHAlVolhrBpXzm/k7eAzdN4fsM2NjCE6YLEI=,2016-04-29 11:04:00,1,1,1184.0


In [23]:
product.head(3)

,Unnamed: 0,TradesGroupCode,SalePageId
0,2.0,Esf9q59OO5QVU6Rapl8m7t8Lyu1MshueQOEyRRXEwU8=,小白鞋
1,5.0,wTDcVew4UUsw/a9ag6EtoIKd4dp+bPyvV3n7jJZag0w=,菱格紋包
2,6.0,3uq44BE2pVcqbv+OgCeNtBQycM8O7Ncy6cqzWo/Wbew=,鞋用配件


In [7]:
df = pd.read_csv('./data.csv')[['UnifiedUserId', 'TradesGroupCode', 'low', 'normal', 'high']]

In [24]:
df.head(3)

,UnifiedUserId,TradesGroupCode,low,normal,high
0,xjOctGt+CvE/+soyAS+LVA==,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,1,1,0
1,Agoqyv2QhggU0nmPX6tVJA==,VTI/4oAuHAlVolhrBpXzm/k7eAzdN4fsM2NjCE6YLEI=,0,1,0
2,xvZqF1Tlc8cGPJQwyRT24A==,BmdhnTN8wcHjLbMp8O3awDOib/WiGtv9yazSs3H5Jnk=,1,0,0


## 在每個主單中加入 1)高價位單品數 2)中價位單品數 3)低價位的單品數

所有子單單價的前25%為高價位，中間50%為中價位，後25%為低價位 (不含0圓商品)

In [8]:
combine = pd.merge(order, df, left_on = 'TradesGroupCode', right_on = 'TradesGroupCode', how = 'left')

In [9]:
combine.head(3)

,TradesGroupCode,OrderDateTime,TsCount,Qty,TotalSalesAmount,UnifiedUserId,low,normal,high
0,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,1,-1,-100.0,xjOctGt+CvE/+soyAS+LVA==,1.0,1.0,0.0
1,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,2,2,1284.0,xjOctGt+CvE/+soyAS+LVA==,1.0,1.0,0.0
2,VTI/4oAuHAlVolhrBpXzm/k7eAzdN4fsM2NjCE6YLEI=,2016-04-29 11:04:00,1,1,1184.0,Agoqyv2QhggU0nmPX6tVJA==,0.0,1.0,0.0


In [10]:
combined = pd.merge(combine, member, left_on = 'UnifiedUserId', right_on = 'UnifiedUserId', how = 'left')

In [11]:
combined = combined[combined['Qty'] >= 0]

In [12]:
combined.head()

,TradesGroupCode,OrderDateTime,TsCount,Qty,TotalSalesAmount,UnifiedUserId,low,normal,high,Gender,Birthday,MemberCardLevel
1,5uPmVe+VOTmLhDIRYqZ0X9WEZkAr2lq1GaDvxM2C/fw=,2016-04-29 09:56:00,2,2,1284.0,xjOctGt+CvE/+soyAS+LVA==,1.0,1.0,0.0,NaN,1900-01-01,10.0
2,VTI/4oAuHAlVolhrBpXzm/k7eAzdN4fsM2NjCE6YLEI=,2016-04-29 11:04:00,1,1,1184.0,Agoqyv2QhggU0nmPX6tVJA==,0.0,1.0,0.0,NaN,0001-01-01,10.0
3,BmdhnTN8wcHjLbMp8O3awDOib/WiGtv9yazSs3H5Jnk=,2016-04-29 11:05:00,1,1,1184.0,xvZqF1Tlc8cGPJQwyRT24A==,1.0,0.0,0.0,NaN,1900-01-01,10.0
4,AlDaYlS/Oda/FRWZqGNt1kV4N6UhQ0tNbA2y7lmS5pI=,2016-05-02 10:29:00,1,1,1080.0,R+nvCLvUXeVCC4OYsSjSzA==,0.0,1.0,0.0,Female,1984-06-05,10.0
5,LZqYjv3x9IzUf5jNUtFz/3vijWR+KcTn3RzfTQF2yXs=,2016-05-03 10:29:00,2,2,1774.0,ftOkl2eeDxYeBcSZX3UiTA==,1.0,1.0,0.0,Female,0076-09-25,10.0


In [13]:
l = []
for i in combined['OrderDateTime']:
    l.append(i.split()[0])
combined['OrderDateTime'] = l

In [14]:
ProductTag = list(product["SalePageId"].unique())
print(ProductTag)

['小白鞋', '菱格紋包', '鞋用配件', '短靴', '拖鞋', '贈品', '跟鞋', '婚鞋', '膝靴', 'NG鞋', '身體保養', '鞋墊', '老爹鞋', '涼拖鞋', '穆勒鞋', '牛津鞋', '日用品', '厚底鞋', '尖頭鞋', '帆布鞋', '涼鞋', '長靴', '襪靴', '便鞋', '平底鞋', '小包', '皮夾x卡夾x零錢包', '紳士鞋', '服裝衣物', '懶人鞋', '休閒鞋', '包鞋', '飾品配件', '黛妃包', '包用配件', '凱莉包', '高跟鞋', '草編鞋', '踝靴', '雪靴', '水桶包', '餅乾鞋', '軍靴', '提袋', '尖頭靴', '馬丁靴', '肩背包', '娃娃鞋', '腰包', '低跟鞋', '多功能包', '方包', '雨靴', '方頭鞋', '後背包', '休閒包', '莫卡辛鞋', '劍橋包', '鞋拔', '相機包', 'NG包', '編織包', '球鞋', '腋下包', '樂福鞋', '柏金包', '童鞋', '托特包', '流蘇包', '摺疊鞋', '藤編包', '馬鞍包', '機車靴', '機車包', '工程靴', '慢跑鞋', '帆船鞋', '鍊條包', '拓特包', '短鞋', '跟靴', '蛋糕鞋', '豆豆鞋', '艾瑪包', '情侶鞋', '運動鞋', '瑪麗珍鞋']


In [15]:
combinedWithProduct = product.merge( combined, on = 'TradesGroupCode', how = 'left')

In [16]:
combinedWithProduct.describe()

,Unnamed: 0,TsCount,Qty,TotalSalesAmount,low,normal,high,MemberCardLevel
count,7.310830e+05,731083.000000,731083.000000,731083.000000,731083.000000,731083.000000,731083.000000,731083.000000
mean,1.062197e+06,2.920901,3.066633,2419.015292,1.587338,1.068906,0.358080,14.084912
std,2.784831e+05,2.221187,2.610242,1578.888564,1.860340,0.975659,0.691588,7.352791
min,2.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.295345e+05,1.000000,1.000000,1480.000000,0.000000,0.000000,0.000000,10.000000
50%,1.065257e+06,2.000000,2.000000,1943.000000,1.000000,1.000000,0.000000,10.000000
75%,1.294772e+06,4.000000,4.000000,3000.000000,2.000000,1.000000,1.000000,20.000000
max,1.542842e+06,33.000000,61.000000,50519.000000,26.000000,17.000000,12.000000,40.000000


In [38]:
def AgeCoding(df):
    df.loc[df.Age < 21,'Age']=1
    df.loc[(df.Age > 20) & (df.Age < 31),'Age'] = 2
    df.loc[(df.Age > 30) & (df.Age < 41),'Age'] = 3
    df.loc[(df.Age > 40) & (df.Age < 51),'Age'] = 4
    df.loc[(df.Age > 50) & (df.Age < 61),'Age'] = 5
    df.loc[df.Age > 60,'Age'] = 6
    return df

def GetTag(df, TagList): ##幫每個Tag做Coding
    Dic = {}
    for i in TagList:    
        CodingList = []
        for j in df["SalePageId"]:
            words = j.split("/")
            count = 0
            for k in words:
                if k == i:
                    CodingList.append(1)
                    count += 1
                    break
            if count == 0:
                CodingList.append(0)
        Dic[i] = CodingList
    return Dic

def ModelTraining(df, tags): ##對每個Tag做Training
    score = 0
    for i in tags.keys():
        print("這是 " + i + " 的分類器")
        df_c = df.copy()
        df_c[i] = tags[i]
        
        ListFilyer = ["ReviseMemberCardLevel", "Age", "order", "orderSlave", "low", "normal", "high", "SalesLog"]
        ListFilyer.append(i)
        
        train_data, test_data = train_test_split(df_c[ListFilyer], random_state=777, train_size=0.9)
        ListFilyer.pop()
        X_train = train_data[ListFilyer]
        Y_train = train_data[i]
        X_test = test_data[ListFilyer]
        Y_test = test_data[i]
        
        RF_model = RandomForestClassifier(n_estimators = 100)
        RF_model = RF_model.fit(X_train, Y_train)
        train_score = RF_model.score(X_train, Y_train)
        print("Training score :", train_score)

        Y_predict = RF_model.predict(X_test)
        print('準確率 :', accuracy_score(Y_test, Y_predict))

        conf_mat = confusion_matrix(Y_test, Y_predict)
        print(conf_mat)
        if train_score == 1:
            score += 1
    return score

def Backtesting(df, tags, forwardtest_data, test_tags): ##對每個Tag做Training
    score = 0
    for i in tags.keys():
        print("這是 " + i + " 的分類器")
        df_c = df.copy()
        forwardtest_data_c = forwardtest_data.copy() # added for forwardtest
        df_c[i] = tags[i]
        forwardtest_data_c[i] = test_tags[i] # added for forwardtest
        
        ListFilyer = ["ReviseMemberCardLevel", "Age", "order", "orderSlave", "low", "normal", "high", "SalesLog"]
        ListFilyer.append(i)
        
        train_data, test_data = train_test_split(df_c[ListFilyer], random_state=777, train_size=0.9)
        forwardtest_data_c = forwardtest_data_c[ListFilyer] # added for forwardtest
        ListFilyer.pop()
        X_train = train_data[ListFilyer]
        Y_train = train_data[i]
        X_test = test_data[ListFilyer]
        Y_test = test_data[i]
        X_forward = forwardtest_data_c[ListFilyer] # added for forwardtest
        Y_forward = forwardtest_data_c[i] # added for forwardtest
        
        RF_model = RandomForestClassifier(n_estimators = 100)
        RF_model = RF_model.fit(X_train, Y_train)
#         train_score = RF_model.score(X_train, Y_train)
#         print("Training score :", train_score)

#         Y_predict = RF_model.predict(X_test)
#         print('準確率 :', accuracy_score(Y_test, Y_predict))
        Y_predict = RF_model.predict(X_forward)  # added for forwardtest
        print('後兩年驗證準確率 :', accuracy_score(Y_forward, Y_predict))  # added for forwardtest
        
        conf_mat = confusion_matrix(Y_forward, Y_predict)
        print(conf_mat)
#         if train_score == 1:
#             score += 1
#     return score

### Season 1 (1-3月)(2014-2020)

In [18]:
result = get_season(combinedWithProduct, 2014, 1)
for i in range(2015, 2021,1):
    a = get_season(combinedWithProduct, i, 1)
    result = pd.concat([result, a])
ResultSeason1 = result.dropna()

2014-01-01 2014-04-01
2015-01-01 2015-04-01
2016-01-01 2016-04-01
2017-01-01 2017-04-01
2018-01-01 2018-04-01
2019-01-01 2019-04-01
2020-01-01 2020-04-01


In [19]:
ResultSeason1 = AgeCoding(ResultSeason1)
ResultSeason1["SalesLog"] = np.log(ResultSeason1["sales"])
ResultSeason1["ReviseMemberCardLevel"] = ResultSeason1["MemberCardLevel"] / 10

C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [20]:
TagDictionary = GetTag(ResultSeason1, ProductTag)

In [21]:
NumberOfScore1 = ModelTraining(ResultSeason1, TagDictionary)

這是 小白鞋 的分類器
Training score : 0.9033439180869678
準確率 : 0.8134110787172012
[[2561  202]
 [ 438  229]]
這是 菱格紋包 的分類器
Training score : 1.0
準確率 : 1.0
[[3430]]
這是 鞋用配件 的分類器
Training score : 0.9875251117879593
準確率 : 0.8994169096209913
[[2831  150]
 [ 195  254]]
這是 短靴 的分類器
Training score : 0.9468278141403668
準確率 : 0.8594752186588921
[[2794  164]
 [ 318  154]]
這是 拖鞋 的分類器
Training score : 0.9989955284816279
準確率 : 0.9941690962099126
[[3400    3]
 [  17   10]]
這是 贈品 的分類器
Training score : 0.9609228177046206
準確率 : 0.8772594752186589
[[2410  226]
 [ 195  599]]
這是 跟鞋 的分類器
Training score : 0.9742401658998121
準確率 : 0.9460641399416909
[[3233   42]
 [ 143   12]]
這是 婚鞋 的分類器
Training score : 0.9919318255459789
準確率 : 0.9839650145772595
[[3373    9]
 [  46    2]]
這是 膝靴 的分類器
Training score : 0.9757954766379366
準確率 : 0.9495626822157435
[[3223   42]
 [ 131   34]]
這是 NG鞋 的分類器
Training score : 0.9991899423238935
準確率 : 0.9938775510204082
[[3407    6]
 [  15    2]]
這是 身體保養 的分類器
Training score : 0.9700602682911024
準確率

In [22]:
print("Training Score為1的Tag數:", NumberOfScore1)
print("樣本數不足的Tag")
for i in TagDictionary.keys():
    if sum(TagDictionary[i]) <= 3:
        print(i)

Training Score為1的Tag數: 15
樣本數不足的Tag
菱格紋包
帆布鞋
黛妃包
餅乾鞋
尖頭靴
馬丁靴
方頭鞋
劍橋包
腋下包
藤編包
鍊條包
拓特包
跟靴
蛋糕鞋
豆豆鞋
艾瑪包
瑪麗珍鞋


## 後兩年驗證Season 1 (1-3月)(2021-2022)

In [27]:
test = get_season(combinedWithProduct, 2021, 1)
for i in range(2022, 2023,1):
    a = get_season(combinedWithProduct, i, 1)
    test = pd.concat([test, a])
TestSeason1 = test.dropna()

TestSeason1 = AgeCoding(TestSeason1)
TestSeason1["SalesLog"] = np.log(TestSeason1["sales"])
TestSeason1["ReviseMemberCardLevel"] = TestSeason1["MemberCardLevel"] / 10

TestTagDictionary = GetTag(TestSeason1, ProductTag)

2021-01-01 2021-04-01
2022-01-01 2022-04-01


In [33]:
Backtesting(ResultSeason1, TagDictionary, TestSeason1, TestTagDictionary)

這是 小白鞋 的分類器
後兩年驗證準確率 : 0.8089293145472921
[[13157  1158]
 [ 2056   450]]
這是 菱格紋包 的分類器
後兩年驗證準確率 : 0.9933416562630045
[[16709     0]
 [  112     0]]
這是 鞋用配件 的分類器
後兩年驗證準確率 : 0.8775340348374057
[[14265  1445]
 [  615   496]]
這是 短靴 的分類器
後兩年驗證準確率 : 0.7505499078532787
[[11862  1356]
 [ 2840   763]]
這是 拖鞋 的分類器
後兩年驗證準確率 : 0.9966708281315023
[[16765    11]
 [   45     0]]
這是 贈品 的分類器
後兩年驗證準確率 : 0.8651685393258427
[[9973  759]
 [1509 4580]]
這是 跟鞋 的分類器
後兩年驗證準確率 : 0.9425717852684145
[[15839   247]
 [  719    16]]
這是 婚鞋 的分類器
後兩年驗證準確率 : 0.9762796504369539
[[16419   122]
 [  277     3]]
這是 膝靴 的分類器
後兩年驗證準確率 : 0.942215088282504
[[15767   532]
 [  440    82]]
這是 NG鞋 的分類器
後兩年驗證準確率 : 0.9873967064978301
[[16601    44]
 [  168     8]]
這是 身體保養 的分類器
後兩年驗證準確率 : 0.925450329944712
[[15313   787]
 [  467   254]]
這是 鞋墊 的分類器
後兩年驗證準確率 : 0.8814577016824208
[[14274   893]
 [ 1101   553]]
這是 老爹鞋 的分類器
後兩年驗證準確率 : 0.9695024077046549
[[16302    67]
 [  446     6]]
這是 涼拖鞋 的分類器
後兩年驗證準確率 : 0.9498246239819274
[[15969    94]
 [  

### Season 2 (4-6月)(2014-2019)

In [34]:
result = get_season(combinedWithProduct, 2014, 2)
for i in range(2015, 2020,1):
    a = get_season(combinedWithProduct, i, 2)
    result = pd.concat([result, a])
ResultSeason2 = result.dropna()

2014-04-01 2014-07-01
2015-04-01 2015-07-01
2016-04-01 2016-07-01
2017-04-01 2017-07-01
2018-04-01 2018-07-01
2019-04-01 2019-07-01


In [35]:
ResultSeason2 = AgeCoding(ResultSeason2)
ResultSeason2["SalesLog"] = np.log(ResultSeason2["sales"])
ResultSeason2["ReviseMemberCardLevel"] = ResultSeason2["MemberCardLevel"] / 10


C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [36]:
TagDictionary = GetTag(ResultSeason2, ProductTag)

In [39]:
NumberOfScore1 = ModelTraining(ResultSeason2, TagDictionary)

這是 小白鞋 的分類器
Training score : 0.8806688319499579
準確率 : 0.7961760461760462
[[1922  143]
 [ 422  285]]
這是 菱格紋包 的分類器
Training score : 1.0
準確率 : 1.0
[[2772]]
這是 鞋用配件 的分類器
Training score : 0.9910581819639921
準確率 : 0.9325396825396826
[[2458   68]
 [ 119  127]]
這是 短靴 的分類器
Training score : 0.9912185733188981
準確率 : 0.983044733044733
[[2722   12]
 [  35    3]]
這是 拖鞋 的分類器
Training score : 0.9997995108063675
準確率 : 0.9996392496392497
[[2771    0]
 [   1    0]]
這是 贈品 的分類器
Training score : 0.9796302979269418
準確率 : 0.9116161616161617
[[2078  132]
 [ 113  449]]
這是 跟鞋 的分類器
Training score : 0.9679618268575324
準確率 : 0.9419191919191919
[[2603   26]
 [ 135    8]]
這是 婚鞋 的分類器
Training score : 0.9910180841252656
準確率 : 0.9852092352092352
[[2728    7]
 [  34    3]]
這是 膝靴 的分類器
Training score : 0.9995188259352821
準確率 : 0.9989177489177489
[[2769    0]
 [   3    0]]
這是 NG鞋 的分類器
Training score : 1.0
準確率 : 1.0
[[2772]]
這是 身體保養 的分類器
Training score : 0.9940655198684791
準確率 : 0.961038961038961
[[2639   32]
 [  76   25]]
這

In [40]:
print("Training Score為1的Tag數:", NumberOfScore1)
print("樣本數不足的Tag")
for i in TagDictionary.keys():
    if sum(TagDictionary[i]) <= 3:
        print(i)

Training Score為1的Tag數: 26
樣本數不足的Tag
菱格紋包
NG鞋
老爹鞋
帆布鞋
黛妃包
包用配件
凱莉包
餅乾鞋
尖頭靴
馬丁靴
腰包
方頭鞋
休閒包
劍橋包
相機包
NG包
腋下包
機車靴
拓特包
短鞋
跟靴
蛋糕鞋
豆豆鞋
艾瑪包
情侶鞋
瑪麗珍鞋


## 後兩年驗證Season 2 (4-6月)(2020-2021)

In [41]:
test = get_season(combinedWithProduct, 2020, 2)
for i in range(2021, 2022, 1):
    a = get_season(combinedWithProduct, i, 2)
    test = pd.concat([test, a])
TestSeason2 = test.dropna()

TestSeason2 = AgeCoding(TestSeason2)
TestSeason2["SalesLog"] = np.log(TestSeason2["sales"])
TestSeason2["ReviseMemberCardLevel"] = TestSeason2["MemberCardLevel"] / 10

TestTagDictionary = GetTag(TestSeason2, ProductTag)

Backtesting(ResultSeason2, TagDictionary, TestSeason2, TestTagDictionary)

2020-04-01 2020-07-01
2021-04-01 2021-07-01


C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

這是 小白鞋 的分類器
後兩年驗證準確率 : 0.7422120418848167
[[10465  2211]
 [ 1728   876]]
這是 菱格紋包 的分類器
後兩年驗證準確率 : 0.9981020942408377
[[15251     0]
 [   29     0]]
這是 鞋用配件 的分類器
後兩年驗證準確率 : 0.8768979057591623
[[12933  1075]
 [  806   466]]
這是 短靴 的分類器
後兩年驗證準確率 : 0.9706806282722513
[[14825   124]
 [  324     7]]
這是 拖鞋 的分類器
後兩年驗證準確率 : 0.993586387434555
[[15182     1]
 [   97     0]]
這是 贈品 的分類器
後兩年驗證準確率 : 0.8476439790575916
[[8813  783]
 [1545 4139]]
這是 跟鞋 的分類器
後兩年驗證準確率 : 0.9422774869109948
[[14375   220]
 [  662    23]]
這是 婚鞋 的分類器
後兩年驗證準確率 : 0.9789267015706806
[[14955   156]
 [  166     3]]
這是 膝靴 的分類器
後兩年驗證準確率 : 0.9991492146596859
[[15267     1]
 [   12     0]]
這是 NG鞋 的分類器
後兩年驗證準確率 : 0.9892015706806283
[[15115     0]
 [  165     0]]
這是 身體保養 的分類器
後兩年驗證準確率 : 0.9096858638743456
[[13610   801]
 [  579   290]]
這是 鞋墊 的分類器
後兩年驗證準確率 : 0.8717931937172775
[[12733  1300]
 [  659   588]]
這是 老爹鞋 的分類器
後兩年驗證準確率 : 0.9948298429319372
[[15201     0]
 [   79     0]]
這是 涼拖鞋 的分類器
後兩年驗證準確率 : 0.7879581151832461
[[11710   554]
 [ 

### Season 3 (7-9月)(2014-2019)

In [42]:
result = get_season(combinedWithProduct, 2014, 3)
for i in range(2015, 2020,1):
    a = get_season(combinedWithProduct, i, 3)
    result = pd.concat([result, a])
ResultSeason3 = result.dropna()

2014-07-01 2014-10-01
2015-07-01 2015-10-01
2016-07-01 2016-10-01
2017-07-01 2017-10-01
2018-07-01 2018-10-01
2019-07-01 2019-10-01


In [43]:
ResultSeason3 = AgeCoding(ResultSeason3)
ResultSeason3["SalesLog"] = np.log(ResultSeason3["sales"])
ResultSeason3["ReviseMemberCardLevel"] = ResultSeason3["MemberCardLevel"] / 10

C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [44]:
TagDictionary = GetTag(ResultSeason3, ProductTag)

In [45]:
NumberOfScore1 = ModelTraining(ResultSeason3, TagDictionary)

這是 小白鞋 的分類器
Training score : 0.852050231804474
準確率 : 0.7464560550830296
[[1465  245]
 [ 381  378]]
這是 菱格紋包 的分類器
Training score : 1.0
準確率 : 1.0
[[2469]]
這是 鞋用配件 的分類器
Training score : 0.9889273979385156
準確率 : 0.9238558120696638
[[2193   62]
 [ 126   88]]
這是 短靴 的分類器
Training score : 0.9866768690642301
準確率 : 0.9704333738355609
[[2395   12]
 [  61    1]]
這是 拖鞋 的分類器
Training score : 0.9998199576900572
準確率 : 0.9995949777237748
[[2468    0]
 [   1    0]]
這是 贈品 的分類器
Training score : 0.9843813296124589
準確率 : 0.9307411907654921
[[1367  119]
 [  52  931]]
這是 跟鞋 的分類器
Training score : 0.9672773101678894
準確率 : 0.9303361684892669
[[2292   33]
 [ 139    5]]
這是 婚鞋 的分類器
Training score : 0.9910428950803439
準確率 : 0.9761036857027137
[[2408    9]
 [  50    2]]
這是 膝靴 的分類器
Training score : 0.9987397038304001
準確率 : 0.9979748886188741
[[2464    0]
 [   5    0]]
這是 NG鞋 的分類器
Training score : 0.9994148624926857
準確率 : 0.9910895099230458
[[2444    8]
 [  14    3]]
這是 身體保養 的分類器
Training score : 0.9952738893640005
準確率 

In [46]:
print("Training Score為1的Tag數:", NumberOfScore1)
print("樣本數不足的Tag")
for i in TagDictionary.keys():
    if sum(TagDictionary[i]) <= 3:
        print(i)

Training Score為1的Tag數: 21
樣本數不足的Tag
菱格紋包
老爹鞋
帆布鞋
黛妃包
凱莉包
餅乾鞋
尖頭靴
馬丁靴
方頭鞋
劍橋包
相機包
NG包
腋下包
拓特包
短鞋
跟靴
蛋糕鞋
豆豆鞋
艾瑪包
情侶鞋
瑪麗珍鞋


## 後兩年驗證Season 3 (7-9月)(2020-2021)

In [47]:
test = get_season(combinedWithProduct, 2020, 3)
for i in range(2021, 2022, 1):
    a = get_season(combinedWithProduct, i, 3)
    test = pd.concat([test, a])
TestSeason3 = test.dropna()

TestSeason3 = AgeCoding(TestSeason3)
TestSeason3["SalesLog"] = np.log(TestSeason3["sales"])
TestSeason3["ReviseMemberCardLevel"] = TestSeason3["MemberCardLevel"] / 10

TestTagDictionary = GetTag(TestSeason3, ProductTag)

Backtesting(ResultSeason3, TagDictionary, TestSeason3, TestTagDictionary)

2020-07-01 2020-10-01
2021-07-01 2021-10-01


C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

這是 小白鞋 的分類器
後兩年驗證準確率 : 0.7201648660463373
[[10604  2948]
 [ 1533   928]]
這是 菱格紋包 的分類器
後兩年驗證準確率 : 0.996065696621495
[[15950     0]
 [   63     0]]
這是 鞋用配件 的分類器
後兩年驗證準確率 : 0.9040779366764504
[[14195   741]
 [  795   282]]
這是 短靴 的分類器
後兩年驗證準確率 : 0.9517891712983201
[[15231   183]
 [  589    10]]
這是 拖鞋 的分類器
後兩年驗證準確率 : 0.9966901892212577
[[15960     5]
 [   48     0]]
這是 贈品 的分類器
後兩年驗證準確率 : 0.8514332105164554
[[9462 1714]
 [ 665 4172]]
這是 跟鞋 的分類器
後兩年驗證準確率 : 0.9363642040841816
[[14967   251]
 [  768    27]]
這是 婚鞋 的分類器
後兩年驗證準確率 : 0.9808280771872854
[[15698   109]
 [  198     8]]
這是 膝靴 的分類器
後兩年驗證準確率 : 0.9951914069818273
[[15936    10]
 [   67     0]]
這是 NG鞋 的分類器
後兩年驗證準確率 : 0.9624679947542621
[[15349    91]
 [  510    63]]
這是 身體保養 的分類器
後兩年驗證準確率 : 0.9421719852619747
[[14956   227]
 [  699   131]]
這是 鞋墊 的分類器
後兩年驗證準確率 : 0.9093861237744333
[[14254   586]
 [  865   308]]
這是 老爹鞋 的分類器
後兩年驗證準確率 : 0.9910697558233935
[[15870     0]
 [  143     0]]
這是 涼拖鞋 的分類器
後兩年驗證準確率 : 0.782926372322488
[[12506   141]
 [ 3

### Season 4 (10-12月)(2014-2019)

In [48]:
result = get_season(combinedWithProduct, 2013, 4)
for i in range(2014, 2020,1):
    a = get_season(combinedWithProduct, i, 4)
    result = pd.concat([result, a])
ResultSeason4 = result.dropna()

2013-10-01 2014-01-01
2014-10-01 2015-01-01
2015-10-01 2016-01-01
2016-10-01 2017-01-01
2017-10-01 2018-01-01
2018-10-01 2019-01-01
2019-10-01 2020-01-01


In [49]:
ResultSeason4 = AgeCoding(ResultSeason4)
ResultSeason4["SalesLog"] = np.log(ResultSeason4["sales"])
ResultSeason4["ReviseMemberCardLevel"] = ResultSeason4["MemberCardLevel"] / 10

C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [50]:
TagDictionary = GetTag(ResultSeason4, ProductTag)

In [51]:
NumberOfScore1 = ModelTraining(ResultSeason4, TagDictionary)

這是 小白鞋 的分類器
Training score : 0.8982555368434432
準確率 : 0.7938232770946526
[[2572  254]
 [ 467  204]]
這是 菱格紋包 的分類器
Training score : 1.0
準確率 : 1.0
[[3497]]
這是 鞋用配件 的分類器
Training score : 0.9861777509453147
準確率 : 0.9016299685444666
[[2981  158]
 [ 186  172]]
這是 短靴 的分類器
Training score : 0.9173842585237203
準確率 : 0.792107520732056
[[2461  315]
 [ 412  309]]
這是 拖鞋 的分類器
Training score : 0.9996186965778018
準確率 : 0.9971404060623391
[[3487    0]
 [  10    0]]
這是 贈品 的分類器
Training score : 0.9438848463664962
準確率 : 0.8515870746354017
[[1938  352]
 [ 167 1040]]
這是 跟鞋 的分類器
Training score : 0.9698452543611579
準確率 : 0.9307978267086073
[[3250   42]
 [ 200    5]]
這是 婚鞋 的分類器
Training score : 0.9901178863080297
準確率 : 0.9825564769802688
[[3436    9]
 [  52    0]]
這是 膝靴 的分類器
Training score : 0.975882558545963
準確率 : 0.9348012582213325
[[3095  110]
 [ 118  174]]
這是 NG鞋 的分類器
Training score : 0.9997140224333514
準確率 : 0.9948527309122105
[[3478    2]
 [  16    1]]
這是 身體保養 的分類器
Training score : 0.9624733881986591
準確率 :

In [52]:
print("Training Score為1的Tag數:", NumberOfScore1)
print("樣本數不足的Tag")
for i in TagDictionary.keys():
    if sum(TagDictionary[i]) <= 3:
        print(i)

Training Score為1的Tag數: 17
樣本數不足的Tag
菱格紋包
帆布鞋
黛妃包
凱莉包
餅乾鞋
尖頭靴
馬丁靴
方頭鞋
劍橋包
NG包
腋下包
柏金包
藤編包
鍊條包
豆豆鞋
艾瑪包
情侶鞋
瑪麗珍鞋


## 後兩年驗證Season 4 (10-12月)(2020-2021)

In [53]:
test = get_season(combinedWithProduct, 2020, 4)
for i in range(2021, 2022, 1):
    a = get_season(combinedWithProduct, i, 4)
    test = pd.concat([test, a])
TestSeason4 = test.dropna()

TestSeason4 = AgeCoding(TestSeason4)
TestSeason4["SalesLog"] = np.log(TestSeason4["sales"])
TestSeason4["ReviseMemberCardLevel"] = TestSeason4["MemberCardLevel"] / 10

TestTagDictionary = GetTag(TestSeason4, ProductTag)

Backtesting(ResultSeason4, TagDictionary, TestSeason4, TestTagDictionary)

2020-10-01 2021-01-01
2021-10-01 2022-01-01


C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\kmes8\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

這是 小白鞋 的分類器
後兩年驗證準確率 : 0.8206722937953502
[[15945  1547]
 [ 2086   681]]
這是 菱格紋包 的分類器
後兩年驗證準確率 : 0.9924478009773434
[[20106     0]
 [  153     0]]
這是 鞋用配件 的分類器
後兩年驗證準確率 : 0.8798065057505307
[[17176  1678]
 [  757   648]]
這是 短靴 的分類器
後兩年驗證準確率 : 0.677131151586949
[[11300  2371]
 [ 4170  2418]]
這是 拖鞋 的分類器
後兩年驗證準確率 : 0.8212646231304606
[[16638     6]
 [ 3615     0]]
這是 贈品 的分類器
後兩年驗證準確率 : 0.8199318821264623
[[10108  2568]
 [ 1080  6503]]
這是 跟鞋 的分類器
後兩年驗證準確率 : 0.9430870230514833
[[19081   356]
 [  797    25]]
這是 婚鞋 的分類器
後兩年驗證準確率 : 0.9758625795942544
[[19763   238]
 [  251     7]]
這是 膝靴 的分類器
後兩年驗證準確率 : 0.8869144577718545
[[17483  1369]
 [  922   485]]
這是 NG鞋 的分類器
後兩年驗證準確率 : 0.9919048324201589
[[20089    26]
 [  138     6]]
這是 身體保養 的分類器
後兩年驗證準確率 : 0.9122365368478207
[[18192  1044]
 [  734   289]]
這是 鞋墊 的分類器
後兩年驗證準確率 : 0.8875561478848907
[[17258  1136]
 [ 1142   723]]
這是 老爹鞋 的分類器
後兩年驗證準確率 : 0.9732958191421097
[[19712    46]
 [  495     6]]
這是 涼拖鞋 的分類器
後兩年驗證準確率 : 0.9627819734439015
[[19504    22]